In [ ]:
import numpy as np

me=0.511
e=0.303
Z=57


# 定义 2x2 单位矩阵和泡利矩阵
I_2 = np.eye(2, dtype=complex)
I_4 = np.eye(4, dtype=complex)
sigma1 = np.array([[0, 1], [1, 0]], dtype=complex)
sigma2 = np.array([[0, -1j], [1j, 0]], dtype=complex)
sigma3 = np.array([[1, 0], [0, -1]], dtype=complex)

# 定义 Gamma 矩阵
gamma0 = np.block([[I_2, np.zeros((2, 2))], [np.zeros((2, 2)), -I_2]])
gamma1 = np.block([[np.zeros((2, 2)), sigma1], [-sigma1, np.zeros((2, 2))]])
gamma2 = np.block([[np.zeros((2, 2)), sigma2], [-sigma2, np.zeros((2, 2))]])
gamma3 = np.block([[np.zeros((2, 2)), sigma3], [-sigma3, np.zeros((2, 2))]])

def w(theta,phi,s):
    return np.array([[np.exp(-1j*phi/2)*np.cos((theta+(1/2-s))/2)], [np.exp(1j*phi/2)*np.sin((theta+(1/2-s))/2)]], dtype=complex)
def w_dagger(theta,phi,s):
    return np.array([np.exp(1j*phi/2)*np.cos((theta+(1/2-s))/2), np.exp(-1j*phi/2)*np.sin((theta+(1/2-s))/2)], dtype=complex)


def u(epsilon, theta, phi, s):
    res= np.block([[np.sqrt(epsilon + me)*w(theta,phi,s)],[2*np.sqrt(epsilon - me)*s*w(theta,phi,s)]])
    return res  

def u_f(epsilon_f,theta_f,phi_f,s_f):
    res= np.block([np.sqrt(epsilon_f + me)*w_dagger(theta_f,phi_f,s_f),-2*np.sqrt(epsilon_f - me)*s_f*w_dagger(theta_f,phi_f,s_f)])
    return res

def photon_polar(lamb,theta_k,phi_k):
    return np.array([0,
                     1/np.sqrt(2)*(np.cos(theta_k)*np.cos(phi_k)-1j*lamb*np.sin(phi_k)),
                     1/np.sqrt(2)*(np.cos(theta_k)*np.sin(phi_k)-1j*lamb*np.cos(phi_k)),
                     1/np.sqrt(2)*np.sin(theta_k)], dtype=complex)

def four_vec(energy,theta,phi):
    return np.array([energy,energy*np.sin(theta)*np.cos(phi),energy*np.sin(theta)*np.sin(phi),energy*np.cos(theta)])
def three_vec(energy,theta,phi):
    return np.array([energy*np.sin(theta)*np.cos(phi),energy*np.sin(theta)*np.sin(phi),energy*np.cos(theta)])

def four_vec_slash(energy,theta,phi):
    res= four_vec(energy,theta,phi)[0]*gamma0
    res +=-four_vec(energy,theta,phi)[1]*gamma1
    res +=-four_vec(energy,theta,phi)[2]*gamma2
    res +=-four_vec(energy,theta,phi)[3]*gamma3
    return res

def photon_polar_slash(lamb,theta_k,phi_k):
    res= photon_polar(lamb,theta_k,phi_k)[0]*gamma0
    res +=-photon_polar(lamb,theta_k,phi_k)[1]*gamma1
    res +=-photon_polar(lamb,theta_k,phi_k)[2]*gamma2
    res +=-photon_polar(lamb,theta_k,phi_k)[3]*gamma3
    return res
        




def M(epsilon,theta,phi,s,epsilon_f,theta_f,phi_f,s_f,omega,theta_k,phi_k,lamb):
    res=(4*np.pi)**(3/2)*Z*e**3/((three_vec(epsilon_f,theta_f,phi_f)+three_vec(omega,theta_k,phi_k)-three_vec(epsilon,theta,phi))@\
                        (three_vec(epsilon_f,theta_f,phi_f)+three_vec(omega,theta_k,phi_k)-three_vec(epsilon,theta,phi)))*\
                        u_f(epsilon_f,theta_f,phi_f,s_f)@\
                        (1/(2*(omega*epsilon_f-three_vec(omega,theta_k,phi_k)@three_vec(epsilon_f,theta_f,phi_f)))*photon_polar_slash(lamb,theta_k,phi_k)@\
                        (four_vec_slash(epsilon_f,theta_f,phi_f)+four_vec_slash(omega,theta_k,phi_k)+me*I_4)@gamma0\
                        +\
                        1/(-2*(omega*epsilon-three_vec(omega,theta_k,phi_k)@three_vec(epsilon,theta,phi)))*gamma0@\
                        (four_vec_slash(epsilon,theta,phi)-four_vec_slash(omega,theta_k,phi_k)+me*I_4)@\
                        photon_polar_slash(lamb,theta_k,phi_k))@\
                        u(epsilon,theta,phi,s)
    return res






